In [39]:
import numpy as np
import pandas as pd
import math as m
from einops import rearrange, repeat
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from tqdm import tqdm


### Preperation (NOTE: Using meter as unit)

In [40]:
# Utility functions
# convert pointcloud from cartisean coordinate to spherical coordinate
def cart2sph(xyz):
    x = xyz[:,0]
    y = xyz[:,1]
    z = xyz[:,2]
    XsqPlusYsq = x**2 + y**2
    r = np.sqrt(list(XsqPlusYsq + z**2))
    elev = np.arctan2(list(z), np.sqrt(list(XsqPlusYsq)))
    pan = np.arctan2(list(x), list(y))

    output = np.array([r, elev, pan])
    return rearrange(output, 'a b -> b a') #take transpose


def sph2cart(ang):
    ele = ang[:,0]
    pan = ang[:,1]
    x = np.cos(ele)*np.cos(pan)
    y = np.cos(ele)*np.sin(pan)
    z = np.sin(ele)
    output = np.array([x,y,z])
    return rearrange(output, 'a b -> b a') #take transpose

In [41]:
def loadData():
    # Specify the directory path
    dataset_path = 'datasets/testing1'

    # List all files in the specified path, ignoring directories
    files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
    files.sort()

    # read the files
    points_xyz = []
    for s in files:
        path = 'datasets/testing1/' + s
        df = pd.read_csv(path)
        a = df.to_numpy()
        points_xyz.append(a[:,8:11])
    return points_xyz

def prepareData(points_xyz):
    # Find the fiew direction of each points:
    # NOTE: points in spherical coordinate are arranged: [r, elev, pan]
    points_sphere = []
    for points in points_xyz:
        points_sphere.append(cart2sph(points))

    ### Process the data
    # Translation vectors for points in each view, we are using camera centre at first frame as origin of world coordinate
    # NOTE: translation vectors below are found by assuming transformation between frames are translations, and obatined by manually finding corrspondance
    # They are translation of the same corrspondance across different frames
    # HARD CODED HERE
    t0 = np.array([0,0,0])
    t1 = np.array([-0.671,-0.016,0.215])
    t2 = np.array([-1.825,-0.091,0.147])
    t3 = np.array([-2.661,-0.263,0.166])
    t4 = np.array([-3.607,-0.156,0.039])
    translations = [t0, t1, t2, t3, t4]

    # camera centre locations
    centres = [-t for t in translations]
    centres_data = []
    for i,c in enumerate(centres):
        l = len(points_sphere[i])
        temp = np.tile(c, (l, 1))
        centres_data.append(temp)

    # stack the points into one big matrix
    stacked = []
    for i in range(len(points_sphere)):
        temp = np.hstack((points_sphere[i], centres_data[i]))
        stacked.append(temp)

    dataset = np.array([])
    for i in range(len(stacked)):
        if i == 0:
            dataset = stacked[i]
        else:
            dataset = np.vstack((dataset, stacked[i]))
    np.random.shuffle(dataset)

    # Mid pass filter, for distance value between 2 and 50 meter
    mask1 = dataset[:,0] > 2
    dataset = dataset[mask1]
    mask2 = dataset[:,0] < 50
    dataset = dataset[mask2]

    return dataset

In [42]:
class LiDAR_NeRF(nn.Module):
    def __init__(self, embedding_dim_pos = 10, embedding_dim_dir = 4, hidden_dim = 256, device = 'cuda'):
        super(LiDAR_NeRF, self).__init__()
        self.device = device
        self.embedding_dim_dir = embedding_dim_dir
        self.embedding_dim_pos = embedding_dim_pos
        self.block1 = nn.Sequential(
            nn.Linear(embedding_dim_pos * 6 + 3 + embedding_dim_dir * 4 + 2, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(embedding_dim_pos * 6 + 3 + embedding_dim_dir * 4 + 2 + hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),               
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim,1)
        )
        
    @staticmethod
    def positional_encoding(x, L):
        out = [x]
        for j in range(L):
            out.append(torch.sin(2 ** j * x))
            out.append(torch.cos(2 ** j * x))
        return torch.cat(out, dim=1)

    def forward(self, o, d):
        emb_x = self.positional_encoding(o, self.embedding_dim_pos)
        emb_d = self.positional_encoding(d, self.embedding_dim_dir)
        input = torch.hstack((emb_x,emb_d)).to(dtype=torch.float32)
        temp = self.block1(input)
        input2 = torch.hstack((temp, input)).to(dtype=torch.float32) # add skip input
        output = self.block2(input2)
        return output

In [43]:
def lossBCE(rendered_value, actual_value): 
    loss_bce = nn.CrossEntropyLoss()
    loss = loss_bce(rendered_value, actual_value)
    return loss

In [79]:
test = torch.linspace(0,1,100).expand(2,100)
test

tensor([[0.0000, 0.0101, 0.0202, 0.0303, 0.0404, 0.0505, 0.0606, 0.0707, 0.0808,
         0.0909, 0.1010, 0.1111, 0.1212, 0.1313, 0.1414, 0.1515, 0.1616, 0.1717,
         0.1818, 0.1919, 0.2020, 0.2121, 0.2222, 0.2323, 0.2424, 0.2525, 0.2626,
         0.2727, 0.2828, 0.2929, 0.3030, 0.3131, 0.3232, 0.3333, 0.3434, 0.3535,
         0.3636, 0.3737, 0.3838, 0.3939, 0.4040, 0.4141, 0.4242, 0.4343, 0.4444,
         0.4545, 0.4646, 0.4747, 0.4848, 0.4949, 0.5051, 0.5152, 0.5253, 0.5354,
         0.5455, 0.5556, 0.5657, 0.5758, 0.5859, 0.5960, 0.6061, 0.6162, 0.6263,
         0.6364, 0.6465, 0.6566, 0.6667, 0.6768, 0.6869, 0.6970, 0.7071, 0.7172,
         0.7273, 0.7374, 0.7475, 0.7576, 0.7677, 0.7778, 0.7879, 0.7980, 0.8081,
         0.8182, 0.8283, 0.8384, 0.8485, 0.8586, 0.8687, 0.8788, 0.8889, 0.8990,
         0.9091, 0.9192, 0.9293, 0.9394, 0.9495, 0.9596, 0.9697, 0.9798, 0.9899,
         1.0000],
        [0.0000, 0.0101, 0.0202, 0.0303, 0.0404, 0.0505, 0.0606, 0.0707, 0.0808,
         0

In [44]:
def get_sample_positions(origins, angles, ground_truth_distance, num_bins = 100, device = 'cpu'):
    elev = angles[:,0]
    pan = angles[:,1]
    dir_x = torch.tensor(np.cos(elev)*np.cos(pan))
    dir_y = torch.tensor(np.cos(elev)*np.sin(pan))
    dir_z = torch.tensor(np.sin(elev))

    # create a list of magnitudes with even spacing
    t = torch.linspace(0,1, num_bins, device=device).expand(dir_x.shape[0], num_bins)  # [batch_size, num_bins]
    
    # preterb the spacing
    mid = (t[:, :-1] + t[:, 1:]) / 2.
    lower = torch.cat((t[:, :1], mid), -1)
    upper = torch.cat((mid, t[:, -1:]), -1)
    u = torch.rand(t.shape, device = device)
    t = lower + (upper - lower) * u  # [batch_size, nb_bins]
    
    # convert magnitudes into positions by multiplying unit vector in each direction
    t = rearrange(t, 'a b -> b a')  # [num_bins, batch_size]
    pos_x = dir_x*t     # [num_bins, batch_size]
    pos_y = dir_y*t
    pos_z = dir_z*t
    multiplied = rearrange([pos_x,pos_y,pos_z], 'c b n  -> (n b) c')   # [num_bin*batchsize, 3]
    # tile the origin values
    origins_tiled = repeat(origins, 'n c -> (n b) c', b = num_bins) # [num_bin*batch_size, 3]
    pos = torch.tensor(origins_tiled) + multiplied
    # tile the angle for convenience
    angles_tiled = torch.tensor(repeat(angles, 'n c -> (n b) c', b = num_bins))
    return pos, angles_tiled



In [45]:
# returns pytorch tensor of sigmoid of projected SDF
def get_actual_value(sample_positions, gt_distance, num_bins=100):
    # tile distances
    gt_distance_tiled = repeat(gt_distance, 'b -> (b n) 1', n=num_bins)
    # calculate distance from sample_position
    temp = torch.tensor(sample_positions**2)
    pos_distance = torch.sqrt(torch.sum(temp, dim=1, keepdim=True))

    # find the "projected" value
    sigmoid = nn.Sigmoid()
    values = sigmoid(-(pos_distance - gt_distance_tiled))

    return values

In [46]:
with torch.no_grad():
    sigmoid = nn.Sigmoid()
    pos_distance = torch.tensor(100)
    gt_distance = torch.tensor(100) 
    value = sigmoid(-(pos_distance- gt_distance))
    print(value)

tensor(0.5000)


In [ ]:
# sample data for testing
points = loadData()
dataset = prepareData(points)
test_batch = dataset[128:256,:]
ground_truth_distance = test_batch[:,0]
angles = test_batch[:,1:3]
origin = test_batch[:,3:]
pos, ang = get_sample_positions(origin, angles,num_bins=100)

model = LiDAR_NeRF(hidden_dim=256)
rendered = model(pos, ang)
sigmoid = nn.Sigmoid()
rendered_sigmoid = sigmoid(rendered)
temp = torch.zeros_like(pos)
val = (get_actual_value(pos, ground_truth_distance)).to(dtype = torch.float32)


# for x in val:
#     print(x)
# print(min(rendered))
# loss_bce = nn.BCELoss()
# loss = loss_bce(rendered_sigmoid, val)
# print(loss)

In [41]:
def train(model, optimizer, scheduler, dataloader, device = 'cpu', epoch = int(1e5), num_bins = 100):
    training_losses = []
    for _ in tqdm(range(epoch)):
        for batch in dataloader:
            # parse the batch
            ground_truth_distance = batch[:,0]
            angles = batch[:,1:3]
            origin = batch[:,:3:7]
            
            sample_positions, sample_angles = get_sample_positions(origin, angles, num_bins=num_bins)
            
            rendered_value = model(sample_positions.to(device), sample_angles.to(device))
            sigmoid = nn.Sigmoid()
            rendered_value_sigmoid = sigmoid(rendered_value)
            actual_value_sigmoided = (get_actual_value(sample_positions.to(device), ground_truth_distance.to(device))).to(dtype = torch.float32)
            
            # loss = lossBCE(rendered_value, actual_value_sigmoided)  # + lossEikonal(model)
            # loss_bce = nn.CrossEntropyLoss()
            loss_bce = nn.BCELoss()
            loss = loss_bce(rendered_value_sigmoid, actual_value_sigmoided)
            # BCEWithLogitsLoss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            training_losses.append(loss.item())
            # print(loss.item())
            print(loss.item())
        scheduler.step()
    return training_losses
    

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")
points = loadData()
print("loaded data")
data_matrix = prepareData(points)
print("prepared data")
training_dataset = torch.from_numpy(data_matrix)
data_loader = DataLoader(training_dataset, batch_size=1024, shuffle = True)
model = LiDAR_NeRF(hidden_dim=512, embedding_dim_dir=10, device = device).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2, 4, 8, 16], gamma=0.5)
losses = train(model, optimizer, scheduler, data_loader, epoch = 20, device=device)


In [ ]:
### Save the model
torch.save(model.state_dict(), 'version1_trial1.pth')

In [31]:
#### Load the model and try to "visualize" the model's datapoints
model2 = LiDAR_NeRF(hidden_dim=512, embedding_dim_dir=10, device = 'cpu')
model2.load_state_dict(torch.load('/home/ansonhon/anson/thesis/LiDAR_NeRF/local/models/version1_trial1.pth'))
model2.eval()  # Set the model to inference mode

LiDAR_NeRF(
  (block1): Sequential(
    (0): Linear(in_features=105, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
  )
  (block2): Sequential(
    (0): Linear(in_features=617, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [60]:
# sample data for testing
points = loadData()
dataset = prepareData(points)
test_batch = dataset[0:128,:]
ground_truth_distance = test_batch[:,0]
angles = test_batch[:,1:3]
origin = test_batch[:,3:]
pos, ang = get_sample_positions(origin, angles,num_bins=100)
# ang[:,1] += 0.01
model = LiDAR_NeRF(hidden_dim=256)
rendered = model(pos, ang)
sigmoid = nn.Sigmoid()
rendered_sigmoid = sigmoid(rendered)
# pos = torch.zeros_like(pos)
val = (get_actual_value(pos, ground_truth_distance)).to(dtype = torch.float32)

with torch.no_grad():
    pos_tensor = torch.tensor(pos)
    ang_tensor = torch.tensor(ang)
    output = model2(pos_tensor, ang_tensor)

/tmp/ipykernel_7409/2962768184.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  temp = torch.tensor(sample_positions**2)
/tmp/ipykernel_7409/3574676403.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_tensor = torch.tensor(pos)
/tmp/ipykernel_7409/3574676403.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ang_tensor = torch.tensor(ang)


In [63]:
ground_truth_distance

array([10.399214  , 10.91817829, 13.53060871,  6.07898947, 11.7334501 ,
        9.37332625,  8.71858306,  8.13654893,  8.60786704, 11.26260301,
        6.61249818, 24.06703704, 15.54482499,  9.59759753, 27.90561933,
       38.18257097, 49.65299743, 27.85560469, 12.63629208,  9.20088626,
       15.04839681,  8.09782063,  7.96666677,  5.78550798, 37.44105097,
        4.78177721, 13.00769447, 13.22598086, 12.09721827, 10.75251612,
        9.25793713, 24.2170193 , 13.9909485 ,  8.85495899,  9.80638234,
       49.63953668, 21.07629429, 43.70530456,  6.25467492, 33.22301242,
        9.33456179,  6.39301065, 48.02704691, 23.29812176,  4.17582805,
       13.57226488, 18.06122046,  7.88255098, 10.0327647 , 18.31223302,
        5.91224103, 12.04290148, 15.78143721, 12.58434868, 16.93181907,
       10.78562153,  7.13742712, 19.47748405, 23.78903986,  6.67717171,
       11.44818246, 11.44713756, 20.59084911,  4.50917099, 36.92286079,
       15.31491297,  6.34567305, 15.13709847, 31.7577729 ,  6.55

In [61]:
print(pos[0:100,:])

tensor([[  3.1185,  -0.5239,  -0.6558],
        [  3.5806,  -0.6015,  -0.7530],
        [  4.5801,  -0.7695,  -0.9632],
        [  5.0458,  -0.8477,  -1.0611],
        [  5.6898,  -0.9559,  -1.1966],
        [  6.1265,  -1.0293,  -1.2884],
        [  7.3240,  -1.2304,  -1.5403],
        [  7.7609,  -1.3038,  -1.6322],
        [  8.3259,  -1.3987,  -1.7510],
        [  9.4974,  -1.5956,  -1.9973],
        [ 10.1525,  -1.7056,  -2.1351],
        [ 10.6502,  -1.7892,  -2.2398],
        [ 11.1054,  -1.8657,  -2.3355],
        [ 12.0142,  -2.0184,  -2.5266],
        [ 12.3803,  -2.0799,  -2.6036],
        [ 13.4859,  -2.2656,  -2.8361],
        [ 14.3389,  -2.4089,  -3.0155],
        [ 15.0206,  -2.5235,  -3.1589],
        [ 15.5355,  -2.6100,  -3.2672],
        [ 15.9780,  -2.6843,  -3.3602],
        [ 16.9162,  -2.8419,  -3.5576],
        [ 17.9054,  -3.0081,  -3.7656],
        [ 18.4689,  -3.1028,  -3.8841],
        [ 19.0608,  -3.2022,  -4.0086],
        [ 19.8788,  -3.3396,  -4.1806],


In [59]:
print(output[0:200])

tensor([[-6.2275e-01],
        [-1.0635e+00],
        [-1.3142e+00],
        [-1.7705e+00],
        [-2.5884e+00],
        [-3.0063e+00],
        [-3.6417e+00],
        [-4.4365e+00],
        [-4.8669e+00],
        [-5.4820e+00],
        [-6.4632e+00],
        [-6.6659e+00],
        [-7.2081e+00],
        [-8.4017e+00],
        [-8.9906e+00],
        [-9.6592e+00],
        [-1.0285e+01],
        [-1.0843e+01],
        [-1.1799e+01],
        [-1.3038e+01],
        [-1.3060e+01],
        [-1.3554e+01],
        [-1.4500e+01],
        [-1.5559e+01],
        [-1.6112e+01],
        [-1.6187e+01],
        [-1.6843e+01],
        [-1.7287e+01],
        [-1.7841e+01],
        [-1.8358e+01],
        [-1.9024e+01],
        [-1.9419e+01],
        [-2.0323e+01],
        [-2.1107e+01],
        [-2.1104e+01],
        [-2.1432e+01],
        [-2.1914e+01],
        [-2.2501e+01],
        [-2.2534e+01],
        [-2.3445e+01],
        [-2.3725e+01],
        [-2.4523e+01],
        [-2.4248e+01],
        [-2

In [28]:
with torch.no_grad():
    dist = 1 # initial distanc forvisualization
    pos = torch.zeros((100000,3))
    ele = torch.linspace(-0.34, 0.3, 100)
    pan = torch.linspace(-3.14, 3.14, 1000)
    ele_tiled = repeat(ele, 'n -> (r n) 1', r = 1000)
    pan_tiled = repeat(pan, 'n -> (n r) 1', r = 100)
    ang = torch.cat((ele_tiled, pan_tiled), dim=1)

    # direction for each "point" from camera centre
    directions = torch.tensor(sph2cart(np.array(ang)))

    

    # for i in range(50):
    #     output2 = model2(pos, ang)
    #     temp = torch.sign(output2)
    #     pos += directions * dist * temp
    #     # dist /= 2


        


In [29]:
### Save to csv for visualization
df_temp = pd.read_csv('datasets/testing1/testing.csv')
df_temp = df_temp.head(100000)
pos_np = pos.numpy()
df_temp['X'] = pos_np[:,0]
df_temp['Y'] = pos_np[:,1]
df_temp['Z'] = pos_np[:,2]
print(df_temp.head())

   Version  Slot ID  LiDAR Index  Rsvd  Error Code  Timestamp Type  Data Type  \
0        5        7            1     0  0x00000200               0          0   
1        5        7            1     0  0x00000200               0          0   
2        5        7            1     0  0x00000200               0          0   
3        5        7            1     0  0x00000200               0          0   
4        5        7            1     0  0x00000200               0          0   

      Timestamp         X         Y         Z  Reflectivity  Tag  Ori_x  \
0  339330000000 -7.542027 -0.012011 -2.667897            24    0   8128   
1  339330000000 -7.559117 -0.012038 -2.619085            24    0   8132   
2  339330000000 -5.681918 -0.009049 -1.927623            24    0   8137   
3  339330000000 -5.694260 -0.009068 -1.890851            28    0   8132   
4  339330000000 -5.706365 -0.009088 -1.854000            28    0   8132   

   Ori_y  Ori_z  Ori_radius  Ori_theta  Ori_phi  
0  23651   6

In [30]:
df_temp.to_csv('testing.csv', index=False)